In [ ]:
# Convolutional Neural Networks
"""
Differs by having convolutional layers that pick out patterns in an image
Convolutional layers perform a "convolution operation"
Apply filters that detect patterns.
patterns can include: edges, corners, circles, squares
As go into deeper layers, detect more complex features (eyes, scales, ears)
As go even deeper, may detect objects such as animals

A small grid will slide across image to analyse each section seperately
This is called "convolving"

dot product of image and filter (filter may be a 3x3 grid of random numbers to start)
after input is convolved, end up with another image that is passed to the next layer
These filters are pattern detectors
example filter that detects a top edge:
-1 -1 -1
 1  1  1
 0  0  0
 
"""

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Device:",device)

Device: cuda


In [2]:
# Our neural net will set up 3 convolutional layers and 2 linear layers
# It will run on my local NVIDIA GPU

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) # input, outfeatures, kernel
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
               
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 =  nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        # in order to determine # of features to pass to Linear layer
        # have to flatted and get the shape of the first dimension.
        # assign this to an internal variable to use when
        # defining input for 
        x = torch.flatten(x, 1, -1)
        if self._to_linear is None: 
            self._to_linear = x.shape[1]
        return x
        
    def forward(self, x):
        x = self.convs(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1) # dim 0 are the batches

In [3]:
# Load the training data
import numpy as np
train_data = np.load("train_data.npy", allow_pickle=True)
test_data = np.load("test_data.npy", allow_pickle=True)

X_train = torch.Tensor([i[0] for i in train_data]).view(-1, 50, 50)
X_train = (X_train/255.0) # scale pixels between [0, 1]
y_train = torch.Tensor([i[1] for i in train_data])

X_test = torch.Tensor([i[0] for i in test_data]).view(-1, 50, 50)
X_test = (X_test/255.0) # scale pixels between [0, 1]
y_test = torch.Tensor([i[1] for i in test_data])

In [4]:
# Functions to run the training and testing of the network

def fwd_pass(X, y, train=False):
    if train:
        net.zero_grad()
    outputs = net(X.to(device))
    matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y.to(device))
    
    if train:
        loss.backward()
        optimizer.step()
    return acc, loss

def test(size=32):
    random_start = np.random.randint(len(X_test)-size)
    X, y = X_test[random_start:random_start+size], y_test[random_start:random_start+size]
    with torch.no_grad():
        acc, loss = fwd_pass(X.view(-1, 1, 50, 50), y)
    return acc, loss

In [6]:
# Train and test the model 

import time
# Set up the optimizer
from tqdm import tqdm
import torch.optim as optim
MODEL_NAME = f"conv-32-64-128-lin-512-2-model-{int(time.time())}"

net = Net().to(device)

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss() # mean square error loss

print(MODEL_NAME)

def train(batch_size=100, epochs=3, test_freq=5):
    with open("convnet_logs/"+MODEL_NAME+".log", "a") as f:
        f.write("modelname,time,epoch,accuracy,loss,test_accuracy,test_loss\n")
        for epoch in range(epochs):
            # X_temp, y_temp = unison_shuffled_copies(X_train, y_train)
            for i in tqdm(range(0, len(X_train), batch_size)):
                batch_X = X_train[i:i+batch_size].view(-1, 1, 50, 50)
                batch_y = y_train[i:i+batch_size]
                acc, loss = fwd_pass(batch_X, batch_y, train=True)
                if i % (test_freq*batch_size) == 0:
                    test_acc, test_loss = test()
                    f.write(
                        (
                        f"{MODEL_NAME},{round(time.time(), 3)},{epoch},"
                        f"{round(acc, 2)},{round(float(loss),4)},"
                        f"{round(test_acc, 2)},{round(float(test_loss),4)}\n"
                        )
                    )

train()

        

  1%|█                                                                                 | 3/225 [00:00<00:07, 27.75it/s]

conv-32-64-128-lin-512-2-model-1588012303


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [00:07<00:00, 30.38it/s]


In [7]:
torch.save(net, 'trained_convnn.pt')

c:\users\tom\appdata\local\programs\python\python37\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [8]:
net.save_state_dict('trained_convnn_3epochs.pt')

AttributeError: 'Net' object has no attribute 'save_state_dict'